# Classification Challenge Mark 1

## Given Template

### data.py

In [ ]:
from torch.utils.data import Dataset


class ChristmasImages(Dataset):
    
    def __init__(self, path, training=True):
        super().__init__()
        self.training = training
        # If training == True, path contains subfolders
        # containing images of the corresponding classes
        # If training == False, path directly contains
        # the test images for testing the classifier
        pass
    
    def __getitem__(self, index):
        # If self.training == False, output (image, )
        # where image will be used as input for your model
        raise NotImplementedError


### model.py

In [ ]:
import torch
import torch.nn as nn


class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #############################
        # Initialize your network
        #############################
        
    def forward(self, x):
        
        #############################
        # Implement the forward pass
        #############################
        
        pass
    
    def save_model(self):
        
        #############################
        # Saving the model's weitghts
        # Upload 'model' as part of
        # your submission
        # Do not modify this function
        #############################
        
        torch.save(self.state_dict(), 'model.pkl')

## CODE STARTS HERE

In [1]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models
from functions import *

In [2]:
# Train dataset path
train_path = "/mnt/datasets/deep_learning_challenge/train/"
# Val dataset path
val_path = "/mnt/datasets/deep_learning_challenge/val/"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
class ChristmasImages(Dataset) :

    def __init__(self, path, training=True):
        super().__init__()
        self.training = training
        # If training == True, path contains subfolders
        # containing images of the corresponding classes
        # If training == False, path directly contains
        # the test images for testing the classifier
        
        # The path to the dataset
        self.path = path
        
        # The transformations to be applied to the images
        self.transform = transforms.Compose([transforms.Resize((224,224)),
                                             transforms.RandomHorizontalFlip(p=0.5),
                                             transforms.RandomVerticalFlip(p=0.5),
                                             transforms.ToTensor()])
        
        if(self.training):
            # Creating an ImageFolder dataset
            self.data = datasets.ImageFolder(self.path,transform=self.transform)
        
        else:
            # Directly loading the images from the path
            # Getting the paths of all images in the test directory
            image_paths = [self.path+'/'+ i for i in os.listdir(self.path)]
            # Applying transformations to each test image and stack them into a tensor
            image_list = [self.transform(Image.open(image).convert('RGB')) for image in image_paths]
            self.data = torch.stack(image_list,dim=0)
                

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # If self.training == False, output (image, )
        # where image will be used as input for your model
        
        if(self.training):
            return (self.data[index][0], self.data[index][1]) # Returning the image and its corresponding label
        else:
            return (self.data[index],) # Returning only the transformed image

## Googlenet

In [5]:
class NetworkGooglenet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #############################
        # Initialize your network
        #############################
        
        # Loading the pre-trained GoogLeNet model with default weights
        self.base_model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
        
        # Extracting the feature extraction layers from the GoogLeNet model
        self.features = nn.ModuleList(self.base_model.children())[:-1]
        self.features = nn.Sequential(*self.features)
        
        # Freezing the parameters of the base model to prevent them from being updated during training
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        # Getting the number of input features for the fully connected layer
        fc_inputs = self.base_model.fc.in_features
        
        # Defining the architecture of the fully connected layers
        self.flat = nn.Flatten()
        self.linear1 = nn.Linear(fc_inputs, 256)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(256,8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        #############################
        # Implement the forward pass
        #############################
        x = self.features(x)
        x = self.flat(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    
    def save_model(self):
        
        #############################
        # Saving the model's weitghts
        # Upload 'model' as part of
        # your submission
        # Do not modify this function
        #############################
        
        torch.save(self.state_dict(), 'model.pkl')

In [6]:
gnet = NetworkGooglenet()
device = "cpu"

In [7]:
gnet.load_state_dict(torch.load("gnet_epoch_35.pkl"))

<All keys matched successfully>

In [8]:
train_data = ChristmasImages(train_path,True)
# val_data = ChristmasImages(val_path,False)

In [9]:
batch_size = 342
epochs = 10

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gnet.parameters(),lr=0.001)

In [11]:
train_dataloader = DataLoader(train_data,batch_size,shuffle=True)

In [ ]:
gnet_res = training_data(gnet,train_dataloader,epochs,loss_fn,optimizer,device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.4023 | train_acc: 0.8755 | 
Epoch: 2 | train_loss: 1.4025 | train_acc: 0.8753 | 
Epoch: 3 | train_loss: 1.3957 | train_acc: 0.8814 | 
Epoch: 4 | train_loss: 1.3973 | train_acc: 0.8800 | 
Epoch: 5 | train_loss: 1.3972 | train_acc: 0.8817 | 
Epoch: 6 | train_loss: 1.4038 | train_acc: 0.8725 | 


In [ ]:
plot_loss_curves(gnet_res)

In [ ]:
gnet.save_model()

## Inception V3

In [ ]:
train_data = ChristmasImages(train_path,True)
val_data = ChristmasImages(val_path,False)

In [ ]:
class NetworkInceptionV3(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #############################
        # Initialize your network
        #############################
        self.base_model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        self.features = nn.ModuleList(self.base_model.children())[:-1]
        self.features = nn.Sequential(*self.features)
        for param in self.base_model.parameters():
            param.requires_grad = False
        fc_inputs = self.base_model.fc.in_features
        self.flat = nn.Flatten()
        self.linear1 = nn.Linear(512, 128)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128,8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        #############################
        # Implement the forward pass
        #############################
        x = self.features(x)
        x = self.flat(x)
        # x = self.relu1(self.linear1(x))
        x = self.softmax(self.linear2(x))
        return x
    
    def save_model(self):
        
        #############################
        # Saving the model's weitghts
        # Upload 'model' as part of
        # your submission
        # Do not modify this function
        #############################
        
        torch.save(self.state_dict(), 'model.pkl')

In [ ]:
incept_net = NetworkInceptionV3()
device = "cpu"

In [ ]:
batch_size = 128
epochs = 15
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(incept_net.parameters(),lr=0.001)

In [ ]:
train_dataloader = DataLoader(train_data,batch_size,shuffle=True)

In [ ]:
incept_net_res = training_data(incept_net,train_dataloader,epochs,loss_fn,optimizer)

In [ ]:
# For inception padding error

## VGG

In [ ]:
train_data = ChristmasImages(train_path,True)
val_data = ChristmasImages(val_path,False)

In [ ]:
class NetworkVGG(nn.Module):

    def __init__(self):
        super().__init__()

        #############################
        # Initialize your network
        #############################
        self.base_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

        for param in self.base_model.parameters():
            param.requires_grad = False
        # fc_inputs = self.base_model.fc.in_features
        self.base_model.classifier[-1] = nn.Linear(4096,128)
        # self.linear1 = nn.Linear(, 256)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128,8)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        #############################
        # Implement the forward pass
        #############################
        x = self.base_model(x)
        x = self.softmax(x)
        return x

    def save_model(self):

        #############################
        # Saving the model's weitghts
        # Upload 'model' as part of
        # your submission
        # Do not modify this function
        #############################

        torch.save(self.state_dict(), 'model.pkl')

In [ ]:
vgg_model = NetworkVGG()
device = "cpu"
vgg_model.to(device)

In [ ]:
train_dataloader = DataLoader(train_data,batch_size,shuffle=True)

In [ ]:
batch_size = 512
epochs = 10

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg_model.parameters(),lr=0.001)

In [ ]:
# Second reun for 15 epochs

In [ ]:
vgg_model.load_state_dict(torch.load("vgg_epoch_31.pkl"))

In [ ]:
vgg_res = training_data(vgg_model,train_dataloader,epochs,loss_fn,optimizer,device)

In [ ]:
vgg_model.save_model()